0. Imports

In [12]:
import pandas as pd

1. Load csv

In [13]:
# change to your file location
df = pd.read_csv('/content/drive/MyDrive/Škola/DM/spravanie_zamestnancov_v_zavislosti_od_casu_II/MLM_ZAM_stats.csv', ';', usecols=range(0,10))
df.head(10)

,Effect,Intercept,HODINA,HODINA_STV,PO,UT,STR,STVR,PIA,SO
0,uvod_ZAM,5.58384,-0.68231,0.02633,0.16019,0.00955,-0.76405,-0.98585,0,0
1,studium_ZAM,0.92959,0.06409,-0.00612,0.38236,0.82227,-0.17730,-0.00245,0,0
2,oznamy_ZAM,-3.91218,0.40371,-0.02517,2.31863,2.96682,0.57197,-0.49513,0,0


2. Create collection of weekdays

In [14]:
days = ['PO', 'UT', 'STR', 'STVR', 'PIA']

3. Create logits estimate

In [15]:
dataframe_collection = {} 
index = 0

# Cycle through weekdays
for day in days:

      # Create empty dataframe for weekday
      df_day = pd.DataFrame()

      # Cycle through hours from 7 to 23
      for x in range (7,23):

        # Create logits estimates
        logit_uvod = df.at[index, 'Intercept'] + df.at[index, 'HODINA']*x+df.at[index, 'HODINA_STV']*(x*x)+df.at[index, day]
        logit_studium = df.at[index+1, 'Intercept'] + df.at[index+1, 'HODINA']*x+df.at[index+1, 'HODINA_STV']*(x*x)+df.at[index+1, day]
        logit_oznamy = df.at[index+2, 'Intercept'] + df.at[index+2, 'HODINA']*x+df.at[index+2, 'HODINA_STV']*(x*x)+df.at[index+2, day]

        # Create new row and append it to dataframe
        new_row = {'0_hod': x, '1_uvod': logit_uvod, '2_studium':logit_studium, '3_oznamy':logit_oznamy}
        df_day = df_day.append(new_row, sort=False, ignore_index=True)

      # add dataframe to dictionary of dataframes
      dataframe_collection[day] = df_day

4. Export to excel

In [16]:
# Creating Excel Writer Object from Pandas  
writer = pd.ExcelWriter('ZAM_logits.xlsx',engine='xlsxwriter')   
workbook=writer.book
worksheet=workbook.add_worksheet('ZAM')
writer.sheets['ZAM'] = worksheet

i = 0
for day in days:
  worksheet.write(0, i, str(day + " - ODHAD LOGITOV"))
  dataframe_collection[day].to_excel(writer, sheet_name='ZAM',startrow=1 , startcol=i, index=False)
  i = i+5
writer.save()